In [1]:
import kgbench as kg

data = kg.load('dmg777k') # Load with numpy arrays, and train/validation split

# data = kg.load('dmg777k', torch=True) # Load with pytorch arrays

# data = kg.load('dmg777k', final=True) # Load with numpy arrays and train/test split


100% [..................................................] 397394415 / 397394415loaded data dmg777k (41.98s).


In [2]:
print(data.num_classes)
data.datatypes()

5


['iri',
 'none',
 '@es',
 '@fy',
 '@nl',
 '@nl-nl',
 '@pt',
 '@ru',
 'http://kgbench.info/dt#base64Image',
 'http://www.opengis.net/ont/geosparql#wktLiteral',
 'http://www.w3.org/2001/XMLSchema#anyURI',
 'http://www.w3.org/2001/XMLSchema#boolean',
 'http://www.w3.org/2001/XMLSchema#gYear',
 'http://www.w3.org/2001/XMLSchema#nonNegativeInteger',
 'http://www.w3.org/2001/XMLSchema#positiveInteger']

In [ ]:
print(data.)

In [3]:
data.withheld

array([[288312,      3],
       [287116,      0],
       [287208,      1],
       ...,
       [287084,      3],
       [286606,      1],
       [288132,      1]], dtype=int32)

In [4]:
def create_uri_classes_lists(data, data_set):
    list_of_uris = []
    list_of_classes=[]
    for data_instance in data_set:
    #     print(data_instance)
        list_of_uris.append(data.i2e[data_instance[0]][0])
        list_of_classes.append(data_instance[1])
    return list_of_uris, list_of_classes

In [5]:
list_of_uris_train, list_of_classes_train = create_uri_classes_lists(data, data.training)

In [6]:
list_of_uris, list_of_classes = create_uri_classes_lists(data, data.withheld)

In [7]:
def show_lists(list1, list2):
    for i, (uri, y) in enumerate(zip(list1, list2)):
        print(uri, y)
        if i > 1:
            break
    print(len(list1), len(list2))
    
show_lists(list_of_uris, list_of_classes)
show_lists(list_of_uris_train, list_of_classes_train)


https://data.labs.pdok.nl/rce/id/monument/8800 3
https://data.labs.pdok.nl/rce/id/monument/529502 0
https://data.labs.pdok.nl/rce/id/monument/532 1
1001 1001
https://data.labs.pdok.nl/rce/id/monument/36903 1
https://data.labs.pdok.nl/rce/id/monument/24531 4
https://data.labs.pdok.nl/rce/id/monument/27163 1
5394 5394


In [8]:
data.i2e[286290]

('https://data.labs.pdok.nl/rce/id/monument/519146', 'iri')

In [9]:
import time

In [10]:
import rdflib

start_time = time.time()
g = rdflib.Graph()
g.parse("data/dmg777k_stripped_without_thumbs_geo-.nt")
print("--- %s seconds ---" % (time.time() - start_time))

Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#date, Converter=<function parse_date at 0x7f9ca4f30dc0>
Traceback (most recent call last):
  File "/Users/roderickvanderweerdt/opt/anaconda3/envs/pykeen_env/lib/python3.9/site-packages/rdflib/term.py", line 2084, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
  File "/Users/roderickvanderweerdt/opt/anaconda3/envs/pykeen_env/lib/python3.9/site-packages/isodate/isodates.py", line 201, in parse_date
    return date(sign * int(groups['year']),
ValueError: month must be in 1..12
Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#date, Converter=<function parse_date at 0x7f9ca4f30dc0>
Traceback (most recent call last):
  File "/Users/roderickvanderweerdt/opt/anaconda3/envs/pykeen_env/lib/python3.9/site-packages/rdflib/term.py", line 2084, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
  File "/User

Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#date, Converter=<function parse_date at 0x7f9ca4f30dc0>
Traceback (most recent call last):
  File "/Users/roderickvanderweerdt/opt/anaconda3/envs/pykeen_env/lib/python3.9/site-packages/rdflib/term.py", line 2084, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
  File "/Users/roderickvanderweerdt/opt/anaconda3/envs/pykeen_env/lib/python3.9/site-packages/isodate/isodates.py", line 201, in parse_date
    return date(sign * int(groups['year']),
ValueError: year 0 is out of range
Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#date, Converter=<function parse_date at 0x7f9ca4f30dc0>
Traceback (most recent call last):
  File "/Users/roderickvanderweerdt/opt/anaconda3/envs/pykeen_env/lib/python3.9/site-packages/rdflib/term.py", line 2084, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
  File "/User

--- 140.04820895195007 seconds ---


In [26]:
q = """
SELECT DISTINCT ?monument_uri ?val
WHERE {
    ?monument_uri <https://data.labs.pdok.nl/rce/def/bouwjaar> ?val .
}
"""

q_pop = """
SELECT DISTINCT ?monument_uri ?val
WHERE {
    ?monument_uri <http://dbpedia.org/ontology/city> ?city .
    ?city <http://www.geonames.org/ontology#population> ?val .
}
"""

q_code = """
SELECT DISTINCT ?monument_uri ?val
WHERE {
    ?monument_uri <http://dbpedia.org/ontology/codeNationalMonument> ?val .
}
"""

#monument_values = {year: <int>, population: <int>, codeNationalMonument: <int>}

def add_monument_value(monument_val_dict, value_name, graph, query):
    qres = graph.query(query)
    print("query returned", len(qres), "results.")
    new_dict = 0
    existing_dict = 0
    doubles = 0
    for row in qres:
        monument_str = str(row.monument_uri)
        try:
            try:
#                 monument_val_dict[monument_str][value_name] = int((int(monument_val_dict[monument_str][value_name]) + int(row.val))/2)
                if monument_val_dict[monument_str][value_name] > int(row.val):
                    monument_val_dict[monument_str][value_name] = int(row.val)
                doubles += 1
            except:  
                monument_val_dict[monument_str][value_name] = int(row.val)
                existing_dict += 1
        except:
            new_dict += 1
            monument_val_dict[monument_str] = {}
            monument_val_dict[monument_str][value_name] = int(row.val)
    print("values for new monuments:", new_dict, "\nvalues for previous monuments:", existing_dict, "\nmultiple values for same monuments:", doubles)
    print("monument_val_dict now contains values for", len(monument_val_dict), "monuments.\n")
    return monument_val_dict

value_list = ["year", "population", "codeNationalMonument"]
query_list = [q, q_pop, q_code]
monument_val_dict = {}

start_time = time.time()
for value_name, query in zip(value_list, query_list):
    monument_val_dict = add_monument_value(monument_val_dict, value_name, g, query)
print("--- %s seconds ---" % (time.time() - start_time))

query returned 1800 results.
values for new monuments: 1430 
values for previous monuments: 0 
multiple values for same monuments: 370
monument_val_dict now contains values for 1430 monuments.

query returned 5179 results.
values for new monuments: 4325 
values for previous monuments: 854 
multiple values for same monuments: 0
monument_val_dict now contains values for 5755 monuments.

query returned 8396 results.
values for new monuments: 2641 
values for previous monuments: 5755 
multiple values for same monuments: 0
monument_val_dict now contains values for 8396 monuments.

--- 0.9246320724487305 seconds ---


In [14]:
def print_values(val_list, val_dict):
    values = ""
    for val in val_list:
        try:
            values += str(val_dict[val])+','
        except:
            values += ','
    return values[:-1] #remove last ','
        
def write_entities_dmg(list_of_entities, list_of_y, val_list, entity_val_dict, new_fn):
    un_found = [0,0]
    with open(new_fn, 'w') as f:
        f.write("entity,y,"+','.join(val_list)+'\n')
        for entity, y in zip(list_of_entities, list_of_y):
            try:
                f.write(entity+','+str(y)+','+print_values(val_list, entity_val_dict[entity])+'\n')
                un_found[0] += 1
            except:
                f.write(entity+','+str(y)+','+print_values(val_list, {})+'\n')
#                 print(entity+','+str(y)+','+print_values(val_list, {}))
#                 print("no values found for entity:", entity)
                un_found[1] += 1
    print("found values for", un_found[0], "entities")
    print("couldn't find values for", un_found[1], "entities")

In [20]:
write_entities_dmg(list_of_uris_train, 
                   list_of_classes_train, 
                   value_list, 
                   monument_val_dict, 
                   "data/entities_dmg777k.csv")

write_entities_dmg(list_of_uris, 
                   list_of_classes, 
                   value_list, 
                   monument_val_dict, 
                   "data/entities_dmg777k_VALIDATION.csv")


found values for 5394 entities
couldn't find values for 0 entities
found values for 1001 entities
couldn't find values for 0 entities
